In [130]:
#1.Pulls details of banglore restaurants Zomato pagewise to and saves it to a csv.

In [145]:
import requests
from bs4 import BeautifulSoup
import re
import pandas
from collections import OrderedDict

In [146]:
startUrl = "https://www.zomato.com/bangalore/restaurants"

In [147]:
#Used headers/agent because the request was timed out and asking for an agent. 
#Using following code we can fake the agent.

def getSoup(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    response = requests.get(url,headers=headers)
    
    content = response.content
    soup = BeautifulSoup(content,"html.parser")
    print(url)
    return soup

In [148]:
#max pages
def getMaxPages():
    soup = getSoup(startUrl)
    pages = []
    pagesTag = soup.find("div",attrs={"class":"col-l-4 mtop pagination-number"})
    pagesTag_temp = pagesTag.find_all("b",recursive=True)
    
    for p in pagesTag_temp:
        pages.append(p.get_text())
        
    return int(max(pages))


In [149]:

list_dataframes = []
currentpage = 1
maxPages = getMaxPages()
#print(maxPages)
while currentpage <= maxPages:
    
    pageUrl = startUrl+"?page="+str(currentpage)
    soup = getSoup(pageUrl)
    list_rest = soup.find_all("div",attrs={"class": "col-s-12"})
    
    for rest in list_rest:
        dataframe = OrderedDict() #dict

        #restaurant link
        link = rest.find("a",attrs={"class": "result-title hover_feedback zred bold ln24 fontsize0"})
        dataframe["restaurant_link"] = link['href']
        
        soup = getSoup(link['href'])
        
        #Res Id
        res_id = soup.find("head")
        scripts = res_id.select('script')
        for script in scripts:
            if 'window.RES_ID' in script.text:
                ids = re.findall('\d+', script.contents[0])
                dataframe["restaurant_ID"] = ids[0]
        
        #restaurant title
        top_header_title = (soup.find("h1",attrs={"class":"ui res-name mb0 header nowrap"}))
        dataframe["restaurant_name"] = top_header_title.find("a").get_text().strip()

        #locality
        loc_title = (soup.find("div",attrs={"class":"mb5 pt5 clear"}))
        dataframe["locality"] = loc_title.find("a").get_text().strip()

        #category
        res_category_tag = loc_title.find("span",attrs={"class":"res-info-estabs grey-text fontsize3"})
        if res_category_tag is not None:
            res_category = res_category_tag.find("a")
            if res_category is not None:
                dataframe["restaurant_category"] = res_category_tag.find("a").get_text().strip()
            else:
                dataframe["restaurant_category"] = res_category_tag.get_text().strip()#Delivery
        
        #zomato gold
        goldTag = soup.find("div",attrs={"class":"red_res_tag"})
        if goldTag is not None:
            goldTag_temp = goldTag.find("img")
            if goldTag_temp is not None:
                dataframe["zomato_gold"] = "zomato gold"
            else:
                dataframe["zomato_gold"] = "NA"
        else:
            dataframe["zomato_gold"] = "NA"

        #discounts
        discountTag = soup.find("div",attrs={"class": "fontsize flex flex-center-v"})
        if discountTag is not None:
            discountTag_temp = discountTag.find("div",attrs={"class":"pr5"})
            if discountTag_temp is not None:
                dataframe["discounts"] = discountTag_temp.getText().strip()
            else:
                dataframe["discounts"] = "NA"
        else:
            dataframe["discounts"] = "NA"
        
        #photos
        photosTag = soup.find_all("div",attrs={"respageMenuContainer"})
        for photo in photosTag:
            children = photo.findChildren("a",attrs={"item respageMenu-item photosTab restaurant-tab-item-jumbo-track"})
            for child in children:
                reObj = re.compile('\d+')
                dataframe["photos_taken"] = reObj.findall(child.text.strip())[0]

        #rating
        top_header_rating = (soup.find("div",attrs={"class":"col-l-4 tac left"}))
        rating = top_header_rating.find("div",attrs={"class":"rating_hover_popup res-rating pos-relative clearfix mb5"})

        if rating is not None:
            r = rating.get_text().replace('\n',' ').strip()
            ratingList = r.split('/')
            dataframe["rating"] = ratingList[0].strip()
        else:
            dataframe["rating"] = "NA"
        
        print(dataframe["rating"])

        #votes
        top_header_votes = top_header_rating.find("span",attrs={"class":"mt2 mb0 rating-votes-div rrw-votes grey-text fontsize5 ta-right"})
        if top_header_votes is not None:
            reObj = re.compile('\d+')
            dataframe["votes"] = reObj.findall(top_header_votes.get_text())[0]
        else:
            dataframe["votes"] = "NA"

        #cuisines
        info_left = (soup.find("div",attrs={"class":"col-l-1by3 pl0 pr20"}))
        cuisine_info = info_left.find("div",attrs={"class":"res-info-cuisines clearfix"})
        dataframe["cuisines"] = cuisine_info.get_text()

        #Approx cost
        approxCost_tag = info_left.find("div",attrs={"class":"res-info-detail"})
        if approxCost_tag is not None:
            approx_cost = approxCost_tag.find("span",attrs={"tabindex":"0"})
        if approx_cost is not None:
            reObj = re.compile('\d+')
            dataframe["approx_cost_for_2"] = reObj.findall(approx_cost.get_text().replace(',','').strip())[0]
        else:
            dataframe["approx_cost_for_2"] = "NA"
        
        #Opening timings
        info_centre = (soup.find("div",attrs={"class":"col-l-1by3 pl20 pr20"}))
        if info_centre is not None:
            openingtimes = info_centre.find("div",attrs={"class":"res-info-timings"})
            if openingtimes is not None:
                timings = openingtimes.find("div",attrs={"class":"medium"})
                dataframe["opening_timings"] = re.findall(r'\s(\d*\:\d*\s?(?:AM|PM|am|pm|noon|midnight|Hours|hours)|\d*\s?(?:AM|PM|am|pm|noon|midnight|Hours|hours))',timings.get_text())
            else:
                dataframe["opening_timings"] = "NA"
        
        #address
        addressTag = info_centre.find("div",attrs={"class":"mbot0"})
        address = addressTag.find("span")
        if addressTag and address is not None:
            dataframe["address"] = address.get_text()
        else:
            dataframe["address"] = "NA"
        
        #lat-long
        lat_longTag = soup.find("div",attrs={"class":"resmap pos-relative mt5 mb5"})
        if lat_longTag is not None:
            lat_long_scripts = lat_longTag.select('script')
            for script in lat_long_scripts:
                reObj = re.compile('\d*\.\d*')
                lat_lon = reObj.findall(script.contents[0])
                
                if lat_lon is not None:
                    dataframe["latitude"] = lat_lon[0]
                    dataframe["longitude"] = lat_lon[1]
                else:
                    dataframe["longitude"] = dataframe["latitude"] = "NA"

        #more info
        more_info = (soup.find_all("div",attrs={"class":"res-info-highlights"}))
        if more_info == []:
            dataframe["more_info"] = "NA"
        else:
            for child in more_info:
                infos = child.find_all("div",attrs={"class":"res-info-feature-text"})
                infostring = []
                for info in infos:
                    infostring.append(info.get_text())
                dataframe["more_info"] = infostring
            
        #Featured in Collections
        featured_in_tag = soup.find("div",attrs={"class":"mbot0 ptop0"})
        if featured_in_tag is not None:
            featured_in = featured_in_tag.find("div",attrs={"class":"resbox__main--row res-info-group clearfix"})

            if featured_in is not None:
                features = featured_in.find_all("a")
            if features is not None: 
                dataframe["featured_in"] = [ftr.get_text().replace('\ ',' ').strip() for ftr in features]
                print(dataframe["featured_in"])
            else:
                dataframe["featured_in"] = "NA"
        else:
            dataframe["featured_in"] = "NA"
            
            
        #known for
        knownForTag = soup.find_all("div",attrs={"class":"mbot0 ptop0"})
        if knownForTag == []:
            dataframe["known_for"] = "NA"
        else:
            for child in knownForTag:
                known_for_list = child.find_all("div",attrs={"class":"res-info-known-for-text mr5"})
                for known_for in known_for_list:
                    dataframe["known_for"] = known_for.get_text().strip()            

        #loved here food
        mostlovedTags = soup.find_all("div",attrs={"class":"rv_highlights__wrapper mtop0"})
        j = []
        for child in mostlovedTags:
            j = child.find_all("div",attrs={"class":"rv_highlights__section pr10"})
        
        if j == []:
            dataframe["most_liked_Food"] = "NA"
            dataframe["most_liked_Service"] = "NA"
            dataframe["most_liked_Look & Feel"] = "NA"
        else:    
            for op in j:
                mostlovedcategory = op.find("div",attrs={"class":"grey-text"})

                loved_here = op.find("div",attrs={"class":"rv_highlights__score_bar mt5 mb5"})
                blockbg = loved_here.find_all("div",attrs={"class":"block bd-txt-bg"})

                mostLoved = op.find("div",attrs={"class":"fontsize13 ln18"})
                if mostLoved is not None:
                    item = mostLoved.find_all("span")
                    if item is not None:
                        temp = []
                        temp.append(str(5-len(blockbg))+"/5")
                        for i in item:
                            temp.append(i.get_text().replace(',','').strip())
                        dataframe["most_liked_"+mostlovedcategory.get_text()] = temp

        ##reviews
        review_link = link['href']+"/reviews"
        review_soup = getSoup(review_link)

        review_header = (review_soup.find_all("div",attrs={"class":"res-reviews-container res-reviews-area"}))
        review_list = []
        for i in review_header:
            main = []
            rev_details_parent = i.findChildren("div",attrs={'fs12px pbot0 clearfix'})
            for j in rev_details_parent:
                rev_info = []
                rev_info.append(j.time['datetime'])
                if j.find("div",attrs={"class":"positive-sentiment"}):
                    rev_info.append("positive")
                else:
                    rev_info.append("NA")
                    
                if j.find("div",attrs={"class":"negative-sentiment"}):
                    rev_info.append("negative")
                else:
                    rev_info.append("NA")
                    
                main.append(rev_info)
            
            rev_children = i.findChildren("div",attrs={"rev-text mbot0","rev-text-sm mbot0"})
            
            for indx,child in enumerate(rev_children):
                reviewtxt = child.text.replace('Rated\xa0\n','').strip()
                main[indx].append(reviewtxt.replace('\n',''))
                
        dataframe["reviews"] = tuple(main)
        #print(dataframe["reviews"])
        list_dataframes.append(dataframe)

    currentpage += 1
else:
    currentpage = 1
        



https://www.zomato.com/bangalore/restaurants
https://www.zomato.com/bangalore/restaurants?page=1
https://www.zomato.com/bangalore/abs-absolute-barbecues-marathahalli
4.8
['Great Buffets', "Bengaluru's Finest"]
https://www.zomato.com/bangalore/abs-absolute-barbecues-marathahalli/reviews
https://www.zomato.com/bangalore/uru-brewpark-jp-nagar-bangalore
4.3
['Sneak Peek Bengaluru']
https://www.zomato.com/bangalore/uru-brewpark-jp-nagar-bangalore/reviews
https://www.zomato.com/bangalore/the-big-barbeque-marathahalli-bangalore
4.7
['Sneak Peek Bengaluru', 'Corporate Favorites', 'Great Buffets', "Bengaluru's Finest"]
https://www.zomato.com/bangalore/the-big-barbeque-marathahalli-bangalore/reviews
https://www.zomato.com/bangalore/brahma-brews-jp-nagar-bangalore
4.9
["Bengaluru's Finest", 'Outdoor Seating']
https://www.zomato.com/bangalore/brahma-brews-jp-nagar-bangalore/reviews
https://www.zomato.com/bangalore/dr-sheesha-24-hours-gastro-pub-electronic-city-bangalore
4.1
https://www.zomato.com/

https://www.zomato.com/bangalore/glens-bakehouse-indiranagar/reviews
https://www.zomato.com/bangalore/boho-koramangala-5th-block
4.3
['Great Buffets']
https://www.zomato.com/bangalore/boho-koramangala-5th-block/reviews
https://www.zomato.com/bangalore/restaurants?page=4
https://www.zomato.com/bangalore/opus-club-1-sarjapur-road
4.4
['Best of Gold']
https://www.zomato.com/bangalore/opus-club-1-sarjapur-road/reviews
https://www.zomato.com/bangalore/tbc-sky-lounge-marathahalli-bangalore
4.9
['Happy Hours', 'Corporate Favorites', 'Best Bars and Pubs', "Bengaluru's Finest"]
https://www.zomato.com/bangalore/tbc-sky-lounge-marathahalli-bangalore/reviews
https://www.zomato.com/bangalore/bombay-adda-koramangala-7th-block-bangalore
4.5
["Bengaluru's Finest"]
https://www.zomato.com/bangalore/bombay-adda-koramangala-7th-block-bangalore/reviews
https://www.zomato.com/bangalore/xoox-brewmill-koramangala-5th-block-bangalore
4.5
['Microbreweries', 'Best Bars and Pubs', 'Insta-Worthy', "Bengaluru's Fin

https://www.zomato.com/bangalore/vapour-pub-brewery-indiranagar-bangalore/reviews
https://www.zomato.com/bangalore/eat-fit-hsr-bangalore
4.2
['Just Delivery', 'Food Hygiene Rated Restaurants in Bengaluru', 'Healthy Living']
https://www.zomato.com/bangalore/eat-fit-hsr-bangalore/reviews
https://www.zomato.com/bangalore/kazé-1-lavelle-road-bangalore
4.4
['Luxury Dining', 'Pan-Asian Delicacies']
https://www.zomato.com/bangalore/kazé-1-lavelle-road-bangalore/reviews
https://www.zomato.com/bangalore/xochi-bar-and-kitchen-bellandur-bangalore
4.6
['New on Gold', "Bengaluru's Finest", 'Outdoor Seating']
https://www.zomato.com/bangalore/xochi-bar-and-kitchen-bellandur-bangalore/reviews
https://www.zomato.com/bangalore/fennys-lounge-and-kitchen-koramangala-7th-block
4.6
['Beer in a Bar', "Bengaluru's Finest"]
https://www.zomato.com/bangalore/fennys-lounge-and-kitchen-koramangala-7th-block/reviews
https://www.zomato.com/bangalore/lono-indiranagar-bangalore
4.3
['Best Bars and Pubs', 'Sneak Peek B

https://www.zomato.com/bangalore/citrus-the-leela-palace-airport-road/reviews
https://www.zomato.com/bangalore/oota-bangalore-whitefield
4.5
["Bengaluru's Finest"]
https://www.zomato.com/bangalore/oota-bangalore-whitefield/reviews
https://www.zomato.com/bangalore/sattvam-sadashiv-nagar
4.4
['Veggie Friendly', 'Great Buffets']
https://www.zomato.com/bangalore/sattvam-sadashiv-nagar/reviews
https://www.zomato.com/bangalore/boomrang-barxsocial-koramangala-7th-block-bangalore
4.4
['Sneak Peek Bengaluru', 'Insta-Worthy']
https://www.zomato.com/bangalore/boomrang-barxsocial-koramangala-7th-block-bangalore/reviews
https://www.zomato.com/bangalore/kota-kachori-koramangala-6th-block
4.2
['Veggie Friendly', 'For The Family']
https://www.zomato.com/bangalore/kota-kachori-koramangala-6th-block/reviews
https://www.zomato.com/bangalore/echoes-koramangala-koramangala-5th-block-bangalore
4.6
["Bengaluru's Finest"]
https://www.zomato.com/bangalore/echoes-koramangala-koramangala-5th-block-bangalore/revi

https://www.zomato.com/bangalore/misu-st-marks-road
4.5
['Pan-Asian Delicacies', "Bengaluru's Finest"]
https://www.zomato.com/bangalore/misu-st-marks-road/reviews
https://www.zomato.com/bangalore/village-the-soul-of-india-jp-nagar-bangalore
4.0
['Pan-Asian Delicacies', "Bengaluru's Finest"]
https://www.zomato.com/bangalore/village-the-soul-of-india-jp-nagar-bangalore/reviews
https://www.zomato.com/bangalore/by-chance-brigade-road-bangalore
4.4
https://www.zomato.com/bangalore/by-chance-brigade-road-bangalore/reviews
https://www.zomato.com/bangalore/pasta-street-indiranagar
4.6
['Food Hygiene Rated Restaurants in Bengaluru', 'European & Mediterranean', 'Corporate Favorites', "Bengaluru's Finest"]
https://www.zomato.com/bangalore/pasta-street-indiranagar/reviews
https://www.zomato.com/bangalore/pizza-bakery-brew-eatery-indiranagar-bangalore
4.5
['Rooftops', 'European & Mediterranean', 'Pizza Time!', 'Sneak Peek Bengaluru', "Bengaluru's Finest"]
https://www.zomato.com/bangalore/pizza-bake

...

https://www.zomato.com/bangalore/planet-popcorn-indiranagar-bangalore/reviews
https://www.zomato.com/bangalore/kaulige-millet-corner-brigade-road
3.9
https://www.zomato.com/bangalore/kaulige-millet-corner-brigade-road/reviews
https://www.zomato.com/bangalore/shree-balaji-99-variety-dosa-pav-bhaji-marathahalli-bangalore
-
https://www.zomato.com/bangalore/shree-balaji-99-variety-dosa-pav-bhaji-marathahalli-bangalore/reviews
https://www.zomato.com/bangalore/the-waverly-express-whitefield
3.4
https://www.zomato.com/bangalore/the-waverly-express-whitefield/reviews
https://www.zomato.com/bangalore/bangalore-cakes-electronic-city
3.0
https://www.zomato.com/bangalore/bangalore-cakes-electronic-city/reviews
https://www.zomato.com/bangalore/the-cake-shop-whitefield
-
https://www.zomato.com/bangalore/the-cake-shop-whitefield/reviews
https://www.zomato.com/bangalore/restaurants?page=400
https://www.zomato.com/bangalore/akshatha-nati-style-vijay-nagar
3.7
https://www.zomato.com/bangalore/akshatha-n

https://www.zomato.com/bangalore/cafe-hops-jp-nagar/reviews
https://www.zomato.com/bangalore/ice-cream-works-indiranagar
3.8
https://www.zomato.com/bangalore/ice-cream-works-indiranagar/reviews
https://www.zomato.com/bangalore/cake-stop-rajarajeshwari-nagar-bangalore
3.6
https://www.zomato.com/bangalore/cake-stop-rajarajeshwari-nagar-bangalore/reviews
https://www.zomato.com/bangalore/biriyani-paradise-1-yeshwantpur
3.6
https://www.zomato.com/bangalore/biriyani-paradise-1-yeshwantpur/reviews
https://www.zomato.com/bangalore/cafe-olive-planet-yelahanka
3.8
https://www.zomato.com/bangalore/cafe-olive-planet-yelahanka/reviews
https://www.zomato.com/bangalore/balajis-veg-btm
3.9
https://www.zomato.com/bangalore/balajis-veg-btm/reviews
https://www.zomato.com/bangalore/bamboo-heights-btm
3.8
https://www.zomato.com/bangalore/bamboo-heights-btm/reviews
https://www.zomato.com/bangalore/mirchi-house-electronic-city-bangalore
3.3
https://www.zomato.com/bangalore/mirchi-house-electronic-city-bangal

https://www.zomato.com/bangalore/nandini-swadista-loka-rajajinagar-bangalore/reviews
https://www.zomato.com/bangalore/tandoor-box-whitefield
3.9
https://www.zomato.com/bangalore/tandoor-box-whitefield/reviews
https://www.zomato.com/bangalore/restaurants?page=407
https://www.zomato.com/bangalore/the-shawarma-rolls-paratha-kingdom-bannerghatta-road-bangalore
2.5
https://www.zomato.com/bangalore/the-shawarma-rolls-paratha-kingdom-bannerghatta-road-bangalore/reviews
https://www.zomato.com/bangalore/savjis-roll-n-biryani-vijay-nagar
3.9
https://www.zomato.com/bangalore/savjis-roll-n-biryani-vijay-nagar/reviews
https://www.zomato.com/bangalore/talk-over-table-jeevan-bhima-nagar
3.9
https://www.zomato.com/bangalore/talk-over-table-jeevan-bhima-nagar/reviews
https://www.zomato.com/bangalore/hotel-empark-wilson-garden
3.6
https://www.zomato.com/bangalore/hotel-empark-wilson-garden/reviews
https://www.zomato.com/bangalore/sannidhi-grand-kammanahalli
3.8
https://www.zomato.com/bangalore/sannidhi-

https://www.zomato.com/bangalore/fnp-cakes-n-more-koramangala-4th-block-bangalore/reviews
https://www.zomato.com/bangalore/brew-cafe-btm-bangalore
3.4
https://www.zomato.com/bangalore/brew-cafe-btm-bangalore/reviews
https://www.zomato.com/bangalore/shakti-bar-karavali-family-restaurant-uttarahalli-bangalore
-
https://www.zomato.com/bangalore/shakti-bar-karavali-family-restaurant-uttarahalli-bangalore/reviews
https://www.zomato.com/bangalore/udupi-veg-express-whitefield-bangalore
-
https://www.zomato.com/bangalore/udupi-veg-express-whitefield-bangalore/reviews
https://www.zomato.com/planb-eatout-food-truck
3.3
https://www.zomato.com/planb-eatout-food-truck/reviews
https://www.zomato.com/bangalore/atithi-veg-international-airport-bangalore
3.4


IndexError: list index out of range

In [151]:
df = pandas.DataFrame(list_dataframes,columns=list_dataframes[0].keys())
df.to_csv("zomato_extracted_data_csv/zomato_extracted_data"+".csv",index=False)
